In [1]:
import os
import torch
import pandas as pd
import numpy as np
import scanpy as sc
import gseapy as gp
import seaborn as sns

from tqdm import tqdm
from functools import partial
from sklearn.cluster import KMeans
from scvi.model.base._utils import _de_core
from scvi.model._utils import _get_var_names_from_setup_anndata,  scrna_raw_counts_properties
from scripts import constants
from contrastive_vi.model.contrastive_vi import ContrastiveVIModel

from anndata import AnnData
from typing import Dict, Iterable, Optional, Sequence, Union
from scvi._compat import Literal

Number = Union[int, float]

Global seed set to 0
During startup - Warning messages:
1: package ‘methods’ was built under R version 3.6.1 
2: package ‘datasets’ was built under R version 3.6.1 
3: package ‘utils’ was built under R version 3.6.1 
4: package ‘grDevices’ was built under R version 3.6.1 
5: package ‘graphics’ was built under R version 3.6.1 
6: package ‘stats’ was built under R version 3.6.1 


In [2]:
dataset = "haber_2017"

In [3]:
split_key = constants.DATASET_SPLIT_LOOKUP[dataset]["split_key"]
background_value = constants.DATASET_SPLIT_LOOKUP[dataset]["background_value"]

In [4]:
adata = sc.read_h5ad(
    os.path.join(
        constants.DEFAULT_DATA_PATH,
        f"{dataset}/preprocessed/adata_top_2000_genes.h5ad",
    )
)
ContrastiveVIModel.setup_anndata(adata, layer="count")

INFO     No batch_key inputted, assuming all cells are same batch                            
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["count"]                                               
INFO     Successfully registered anndata object containing 7721 cells, 2000 vars, 1 batches, 
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


In [5]:
target_indices = np.where(adata.obs[split_key] != background_value)[0]
target_adata = adata[target_indices]

In [6]:
genes = adata.var.index

In [7]:
seeds = constants.DEFAULT_SEEDS
latent_size = 10
model_list = [
    torch.load(
        os.path.join(
            constants.DEFAULT_RESULTS_PATH,
            f"{dataset}/contrastiveVI/latent_{latent_size}/{seed}/model.ckpt",
        ),
        map_location="cpu",
    )
    for seed in tqdm(seeds)
]
latent_rep_list = [
    np.load(
        os.path.join(
            constants.DEFAULT_RESULTS_PATH,
            f"{dataset}/contrastiveVI/latent_{latent_size}/{seed}/latent_representations.npy",
        ),
    )
    for seed in tqdm(seeds)
]

100%|████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 18.76it/s]


In [8]:
result_list = []
for seed_index, seed in enumerate(seeds):
    model = model_list[seed_index]
    latent_rep = latent_rep_list[seed_index]
    
    target_adata = model._validate_anndata(target_adata.copy())
    col_names = _get_var_names_from_setup_anndata(target_adata)
    
    latent_clusters = KMeans(n_clusters=2, random_state=123).fit(latent_rep).labels_
    target_adata.obs[f"cluster_{seeds[seed_index]}"] = latent_clusters.astype(str)

    def get_normalized_expression(
        adata: Optional[AnnData] = None,
        indices: Optional[Sequence[int]] = None,
        transform_batch: Optional[Sequence[Union[Number, str]]] = None,
        gene_list: Optional[Sequence[str]] = None,
        library_size: Union[float, Literal["latent"]] = 1,
        n_samples: int = 1,
        n_samples_overall: int = None,
        batch_size: Optional[int] = None,
        return_mean: bool = True,
        return_numpy: Optional[bool] = None,
    ):
        exprs = model.get_normalized_expression(
            adata=adata,
            indices=indices,
            transform_batch=transform_batch,
            gene_list=gene_list,
            library_size=library_size,
            n_samples=n_samples,
            n_samples_overall=n_samples_overall,
            batch_size=batch_size,
            return_mean=return_mean,
            return_numpy=return_numpy,
        )
        return exprs["salient"]
    
    model_fn = partial(
        get_normalized_expression,
        return_numpy=True,
        n_samples=100,
        batch_size=128,
    )
    result = _de_core(
        target_adata,
        model_fn,
        groupby=f"cluster_{seeds[seed_index]}",
        group1="0",
        group2="1",
        idx1=None,
        idx2=None,
        all_stats=True,
        all_stats_fn=scrna_raw_counts_properties,
        col_names=col_names,
        mode="change",
        batchid1=None,
        batchid2=None,
        delta=0.25,
        batch_correction=False,
        fdr=0.05,
        silent=False,
    )
    
    result.reset_index()
    result["gene_symbol"] = result.index
    result_list.append(result)

DE...: 100%|████████████████████████████████████████████████████████████████| 1/1 [04:18<00:00, 258.18s/it]


In [9]:
for seed_index, seed in enumerate(seeds):
    result_list[seed_index]["seed"] = seed
all_result = pd.concat(result_list)

In [10]:
all_result_mean = (
    all_result.groupby("gene_symbol", as_index=False)
    .mean()
    .sort_values(by="proba_de", ascending=False)
)

In [11]:
top_genes = all_result_mean[all_result_mean["proba_de"] > 0.95]["gene_symbol"].tolist()

enr = gp.enrichr(
    gene_list=top_genes,
    gene_sets="KEGG_2019_Mouse",
    organism="mouse",
    cutoff=0.5,
)

enr_results = enr.results

In [12]:
cols = ["Gene_set", "Term", "Adjusted P-value", "Overlap", "Genes"]
enr_results[cols]

,Gene_set,Term,Adjusted P-value,Overlap,Genes
0,KEGG_2019_Mouse,Fat digestion and absorption,0.023545,5/40,FABP1;FABP2;PLA2G3;APOA1;APOA4
1,KEGG_2019_Mouse,Vitamin digestion and absorption,0.023545,4/24,CUBN;RBP2;APOA1;APOA4
2,KEGG_2019_Mouse,Cholesterol metabolism,0.029304,5/49,APOH;APOC2;APOA1;APOC3;APOA4
3,KEGG_2019_Mouse,PPAR signaling pathway,0.251684,5/85,FABP1;FABP2;APOA1;APOC3;HMGCS2
4,KEGG_2019_Mouse,Taurine and hypotaurine metabolism,0.253837,2/11,GGT7;GGT1
...,...,...,...,...,...
142,KEGG_2019_Mouse,cAMP signaling pathway,0.976313,1/211,MYL9
143,KEGG_2019_Mouse,Thermogenesis,0.982260,1/231,CNR1
144,KEGG_2019_Mouse,Human cytomegalovirus infection,0.984735,1/255,ELK1
145,KEGG_2019_Mouse,Endocytosis,0.984735,1/269,RAB11FIP5
